## Preprocessing

### Estrazione frasi
Estraiamo un campione di frasi a partire dai file conllu di Wikipedia italiana elaborata. 

In [35]:
import os
import re
import pandas as pd
from tqdm import tqdm


In [36]:
def get_id(line):
    sent_id = re.sub(r'\D', '', line)
    return sent_id

def get_text(line):
    sent = line[9:].rstrip('\n')
    return sent

def get_sentences(file_conllu):
    sentences = []
    ids = []
    with tqdm(total=sum(1 for _ in open(file_conllu, 'r', encoding='utf-8')), desc=f'Progresso estrazione frasi del file {file_conllu}') as pbar:
        for line in open(file_conllu, 'r', encoding='utf-8'):
            pbar.update(1)  # Aggiorna la barra di avanzamento
            if line.startswith("# sent_id"):
                current_id = get_id(line)  
                ids.append(current_id)
            elif line.startswith("# text"):
                current_sent = get_text(line)
                sentences.append(current_sent)
    return ids, sentences

In [37]:
#nel caso in cui dovessimo itereare sui diversi file nella dir
ds_directory = "C:/Users/bergo/OneDrive - University of Pisa/Tesi Magistrale/wiki_conllu"
ds_files = []
for file_name in os.listdir(ds_directory):
    file_path = os.path.join(ds_directory, file_name)
    ds_files.append(file_path)  #ottengo il nome dei diversi file all'interno della directory
print(ds_files)

['C:/Users/bergo/OneDrive - University of Pisa/Tesi Magistrale/wiki_conllu\\wiki_10_2021_015.conllu']


In [38]:
data_df = pd.DataFrame(columns=["id", "text"])
data_df["id"], data_df["text"] = get_sentences(ds_files[0])   #per ora lavoro su un singolo file

Progresso estrazione frasi del file C:/Users/bergo/OneDrive - University of Pisa/Tesi Magistrale/wiki_conllu\wiki_10_2021_015.conllu: 100%|██████████| 29437395/29437395 [00:37<00:00, 781439.83it/s]


In [39]:
data_df.head()

,id,text
0,1,impeto.
1,2,L’occupazione di Pastrengo.
2,3,Oramai per gli austriaci si trattava di prende...
3,4,Gli uomini della Brigata “Cuneo” intanto entra...
4,5,Ma una batteria di artiglieria a cavallo dei p...


In [40]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1057516 entries, 0 to 1057515
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   id      1057516 non-null  object
 1   text    1057516 non-null  object
dtypes: object(2)
memory usage: 16.1+ MB


In [48]:
samplino = data_df.sample(n=2000, random_state=12) #campione piccino per fare le prove

### Estrazione degli indici di leggibilità per ogni frase

Utilizziamo l'API _read-it_ per calcolare gli indici di complessità di ogni frase.

In [42]:
import requests
import sys

In [43]:

SERVER_PATH = "http://api.italianlp.it"
#SERVER_PATH = "http://itanlp-gpu.ilc.cnr.it:13000"

def load_document(text):
    r = requests.post(SERVER_PATH + '/documents/',           # carica il documento nel database del server
                      data={'text': text,                    # durante il caricamento viene eseguita un'analisi linguistica necessaria per calcolare la leggibilità
                          'lang': 'IT',
                          'extra_tasks': ["readability"]     # chiede al server di calcolare anche la leggibilità del docuemnto
                  })
    doc_id = r.json()['id']                                  # id del documento nel database del server, che serve per richiedere i risultati delle analisi
    return doc_id

def get_doc_score(doc_id):
    r = requests.get(SERVER_PATH + '/documents/details/%s' % doc_id)
    result = r.json()
    return result



In [47]:
#facciamo una prova con un testo di poche frasi
text = "La città era avvolta in una leggera foschia mattutina, creando un'atmosfera misteriosa e incantevole. Gli uccelli cantavano melodie dolci mentre il sole iniziava a sorgere all'orizzonte. Maria camminava lentamente lungo il sentiero del parco, godendosi la tranquillità del nuovo giorno."


doc_id = load_document(text)
r_score = get_doc_score(doc_id)

#vediamo com'è strutturato l'oggetto che viene restituito dall'API
print(r_score)

{'named_entity_executed': False, 'doc_time': '2024-05-15T10:24:48.191507Z', 'readability_executed': True, 'sentences': {'count': 3, 'prev': None, 'data': [{'sentiment_positive_negative_probability': None, 'sentiment_value': None, 'sequence': 1, 'readability_score_all': 60.5895926403111, 'sentiment_negative_probability': None, 'sentiment_neutral_probability': None, 'tokens': [{'word': 'La', 'ten': None, 'sequence': 1, 'pos': 'RD', 'per': None, 'lemma': 'il', 'num': 's', 'named_entity_instance': None, 'dep_type': 'det', 'dep_parent_sequence': 2, 'cpos': 'R', 'gen': 'f', 'mod': None}, {'word': 'città', 'ten': None, 'sequence': 2, 'pos': 'S', 'per': None, 'lemma': 'città', 'num': 'n', 'named_entity_instance': None, 'dep_type': 'subj_pass', 'dep_parent_sequence': 4, 'cpos': 'S', 'gen': 'f', 'mod': None}, {'word': 'era', 'ten': 'i', 'sequence': 3, 'pos': 'VA', 'per': '3', 'lemma': 'essere', 'num': 's', 'named_entity_instance': None, 'dep_type': 'aux', 'dep_parent_sequence': 4, 'cpos': 'V', '

In [45]:
#qui iteriamo sui risultati ottenuti per un singolo documento
def get_sen_scores(doc_scores):
    all_sent_rscore = []
    all_sent_text = []
    for i, sent_results in enumerate(doc_scores['sentences']['data']):
        all_sent_rscore.append(sent_results['readability_score_all'])
        all_sent_text.append(sent_results['raw_text'])
    return all_sent_text, all_sent_rscore

#si passa all'api un doc di 1000 frasi per volta.
def readability_extraction(sen_list, x): #x è il numero di frasi che si vogliono calcolare per iterazione 
    all_scores = []
    for i in tqdm(range(0, len(sen_list), x), desc=f"Progresso nell'estrazione degli indici di complessità delle frasi (ogni iterazione corrisponde a {x} frasi)"):
        doc = "\n".join(sen_list[i:i+x])
        doc_id = load_document(doc)
        r_scores = get_doc_score(doc_id)
        all_scores.append(r_scores)
    return all_scores

#qui si ottiene la struttura dati finale dove per ogni frase abbiamo lo score ottenuto da read_it
def get_final_data(all_scores):
    text_list = []
    score_list = []
    for doc_scores in all_scores:
        text, score = get_sen_scores(doc_scores)
        text_list = text_list + text
        score_list = score_list + score
    return text_list, score_list


In [49]:
all_results = readability_extraction(samplino["text"].tolist(), 1000)
all_sent, all_scores = get_final_data(all_results)

Progresso nell'estrazione degli indici di complessità delle frasi (ogni iterazione corrisponde a 1000 frasi): 100%|██████████| 2/2 [05:04<00:00, 152.43s/it]


### Creazione e valutazione del dataset finale

Si ottiene una struttura dataframe con _id_, _testo e _leggibilità_ per ogni frase. Dopodiché valutiamo le caratteristiche dei dati ottenuti

In [50]:
#si crea un dataframe con tutte le informazioni che ci interessano
data_set = pd.DataFrame(columns=["id", "text", "readability"])
data_set["text"] = all_sent
data_set["readability"] = all_scores
data_set["id"] = range(1, len(data_set) + 1)
data_set.head()

,id,text,readability
0,1,Scritto dal 1943 (quando ancora portava il tit...,93.986138
1,2,Marc Edmund Jones,NaN
2,3,In Piemonte il referente era loro cugino Rocco...,29.705289
3,4,Nel suo libro del 1922 Funakoshi nomina il kat...,45.284815
4,5,LeBell è solitamente all'angolo della campione...,28.109959


In [51]:
data_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           2000 non-null   int64  
 1   text         2000 non-null   object 
 2   readability  1528 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 47.0+ KB


Si controlla l'ordine dei dati. Da tenere in considerazione il fatto che "all_results" è in forma di lista contenente i dizionari dei risultati dell'API per ogni "documento" (ovvero ogni x frasi). Quindi l'indice utilizzato per gli elementi di "all_results" per controllare l'ordine sarà sempre compreso tra 0 e x. Mentre quello del nostro dataframe sarà l'equivalente + x*y (dove y, è il numero del documento che stiamo prendendo in considerazione)

In [52]:
#ora controlliamo che l'ordine sia corretto...
tupla = data_set[data_set["id"]==999]
da_result = all_results[0]['sentences']['data'][998] #primo "documento"
print(f"---- Nel dataframe finale ----\n Testo: '{tupla['text'].item()}' - R_score: {tupla['readability'].item()}\n\n") 
print(f"---- Nei risultati ottenuti dall'API ----\n Testo: '{da_result['raw_text']}' - R_score: {da_result['readability_score_all']}")

---- Nel dataframe finale ----
 Testo: 'Durante i combattimenti Cratero fu ferito da una freccia.' - R_score: 5.41290931277294


---- Nei risultati ottenuti dall'API ----
 Testo: 'Durante i combattimenti Cratero fu ferito da una freccia.' - R_score: 5.41290931277294


In [53]:
tupla = data_set[data_set["id"]==1250]
da_result = all_results[1]['sentences']['data'][249] #secondo "documento"
print(f"---- Nel dataframe finale ---- \n Testo: {tupla['text'].item()} - R_score: {tupla['readability'].item()}\n\n") 
print(f"---- Nei risultati ottenuti dall'API ---- \n  Testo: {da_result['raw_text']} - R_score: {da_result['readability_score_all']}")

---- Nel dataframe finale ---- 
 Testo: Alcuni autori hanno scritto che gran parte del razzismo contro le persone LGBT nere è radicato in un razzismo di tipo sessuale con il suo collegamento a stereotipi di ruolo sessuali associati a stereotipi razziali. - R_score: 44.7483680128475


---- Nei risultati ottenuti dall'API ---- 
  Testo: Alcuni autori hanno scritto che gran parte del razzismo contro le persone LGBT nere è radicato in un razzismo di tipo sessuale con il suo collegamento a stereotipi di ruolo sessuali associati a stereotipi razziali. - R_score: 44.7483680128475


In [54]:
valori_nulli = data_set[data_set["readability"].isnull()]
valori_nulli #vediamo che le frasi troppo corte non restituiscono un valore di leggibilità. 

,id,text,readability
1,2,Marc Edmund Jones,NaN
7,8,Il tallero di Sigismondo.,NaN
12,13,È un terzino sinistro.,NaN
22,23,Caratteristiche craniche e dentarie.,NaN
23,24,Storia.,NaN
...,...,...,...
1990,1991,Spinus notatus,NaN
1996,1997,Karl Saur,NaN
1997,1998,Regno Unito.,NaN
1998,1999,GESCAL,NaN


In [55]:
data_set = data_set.dropna()
data_set.to_csv("ds_leggibilita.csv")